<a href="https://colab.research.google.com/github/aelydens/aie-resources/blob/main/AL_RAG_NVIDIA_10k_Filings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NVIDIA 10k Filings RAG with Langchain

Deliverables:

Build 🏗️
* Data: NVIDIA 10-k Filings
* Model: OpenAI text-3-embedding small, GPT-3.5-turbo
* Tooling: LangChain or LlamaIndex (you choose)
* Vector Store: FAISS
* Additional Component: Add one of the following: 1) visibility with WandB OR 2) evaluation with RAGAS
Ship 🚢

Evaluate your answers to the following questions
* "Who is the E-VP, Operations - and how old are they?"
* "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"
* Record <10 min loom video walkthrough

In [21]:
!pip install -qU langchain langchain-core langchain-community langchain-openai pymupdf faiss_cpu

In [22]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


Initialize our embedding model to be `text-embedding-3-small`.

In [23]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Load and split our documents.

In [24]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyMuPDFLoader("https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/1cbe8fe7-e08a-46e3-8dcc-b429fc06c1a4.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

Set up our FAISS-powered vector store, and create a retriever.

In [25]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

retriever = vector_store.as_retriever()

Create our prompt using `ChatPromptTemplate`.

In [26]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

Initialize our OpenAI model with a temperature of 0.

In [27]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

Create our LCEL chain.

In [28]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | openai_chat_model, "context": itemgetter("context")}
)


Question time!

In [31]:
question = "Who is the E-VP, Operations - and how old are they?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Debora Shoquist is the Executive Vice President of Operations, and she is 69 years old.


In [33]:
question = "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023 in billions?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

$3.539 billion
